In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import sys, os

### Load data

In [2]:
laptop_data=pd.read_csv("../../data/laptop_final.csv")
laptop_data

,link,name,Display Size,Resolution: width,Resolution: height,Refresh Rate,Panel Type,Disk Capacity,SSD,HDD,...,GPU: Name,GPU: Process Size (nm),GPU: Architecture,GPU: Memory Type,GPU: Memory Frequency (MHz),GPU: Memory Size,GPU: Base Clock,GPU: Memory Bus,GPU: Release quarter,GPU: Power Consumption
0,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Strix G17,17.3,2560.0,1440.0,240.0,IPS + G-Sync,2000.0,1.0,0.0,...,"geforce rtx 4060 (laptop, 140w)",4.0,Ada Lovelace,DDR6,2000.0,8.0,1920.0,128.0,2023.0,140.0
1,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Zephyrus Duo 16,16.0,2560.0,1600.0,240.0,IPS + FreeSync,16000.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Zephyrus Duo 16,16.0,2560.0,1600.0,240.0,IPS + FreeSync,2000.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Zephyrus Duo 16,16.0,2560.0,1600.0,240.0,IPS + FreeSync,2000.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Strix G17,17.3,2560.0,1440.0,240.0,IPS + G-Sync,2000.0,1.0,0.0,...,"geforce rtx 4060 (laptop, 140w)",4.0,Ada Lovelace,DDR6,2000.0,8.0,1920.0,128.0,2023.0,140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289166,https://laptopmedia.com/laptop-specs/acer-aspi...,Acer Aspire 5,17.3,1920.0,1080.0,60.0,IPS,256.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289167,https://laptopmedia.com/laptop-specs/hp-17-5749/,HP 17,17.3,1600.0,900.0,60.0,TN,256.0,1.0,0.0,...,iris xe graphics g4,10.0,Tiger Lake GT2,shared,0.0,0.0,300.0,0.0,2020.5,15.0
289168,https://laptopmedia.com/laptop-specs/hp-17-17-...,HP 17 (17-by4000),17.3,1600.0,900.0,60.0,TN,2256.0,1.0,1.0,...,iris xe graphics g4,10.0,Tiger Lake GT2,shared,0.0,0.0,300.0,0.0,2020.5,15.0
289169,https://laptopmedia.com/laptop-specs/hp-17-4594/,HP 17,17.3,1600.0,900.0,60.0,TN,1512.0,1.0,1.0,...,iris xe graphics g4,10.0,Tiger Lake GT2,shared,0.0,0.0,300.0,0.0,2020.5,15.0


#### Drop unnecessary columns, duplicates and na

In [3]:
laptop_data2=laptop_data.drop(columns=["link","name","CPU: Codename","GPU: Name","CPU: Name","Ethernet LAN"])
laptop_data2.drop_duplicates(inplace=True)
laptop_data2.dropna(inplace=True)
laptop_data2.shape

(35404, 56)

In [4]:
#create a dataset for inference
data_for_inference=laptop_data.loc[laptop_data2.index]

#### Create some new attributes for serving UI

In [5]:
def reformat_number(n):
    return str(int(n))
def get_display_format():
    display_size=data_for_inference["Display Size"].apply(str)+"inches "
    resolution=data_for_inference["Resolution: width"].apply(reformat_number)+"x"+data_for_inference["Resolution: height"].apply(reformat_number)+ "pixels "
    refresh_rate=data_for_inference["Refresh Rate"].apply(reformat_number)+"Hz"
    panel=data_for_inference["Panel Type"]
    d=display_size+resolution+refresh_rate+panel
    return d
def get_ram():
    return data_for_inference["RAM Capacity"].apply(reformat_number)+"GB "+data_for_inference["RAM Type"]
def get_disk():
    storage_type = data_for_inference.apply(lambda row: '+'.join(col for col in ['SSD', 'HDD', 'Optane'] if row[col] == 1), axis=1)
    return data_for_inference["Disk Capacity"].apply(reformat_number)+"GB "+storage_type

In [6]:
data_for_inference["name_to_display"]=data_for_inference["name"]+" "+data_for_inference["Display Size"].apply(str)+" inches "+data_for_inference["CPU: Name"]+" "+data_for_inference["RAM Capacity"].apply(reformat_number)+"GB - "+data_for_inference["Disk Capacity"].apply(reformat_number)+"GB "+data_for_inference["GPU: Name"]
data_for_inference["Display"]=get_display_format()
data_for_inference["RAM"]=get_ram()
data_for_inference["Disk"]=get_disk()
data_for_inference["Brand"]=data_for_inference["name"].apply(lambda x:x.split()[0])

In [7]:
data_for_inference["name_to_display"].value_counts()

name_to_display
HP 15 15.6 inches core i5-1135g7 32GB - 1000GB iris xe graphics g7 (80eu)                  118
HP 15 15.6 inches core i5-1135g7 16GB - 512GB iris xe graphics g7 (80eu)                   108
HP 15 15.6 inches core i5-1135g7 16GB - 1000GB iris xe graphics g7 (80eu)                   95
HP 17 17.3 inches core i5-1135g7 32GB - 1000GB iris xe graphics g7 (80eu)                   90
HP 15 15.6 inches core i5-1135g7 8GB - 256GB iris xe graphics g7 (80eu)                     77
                                                                                          ... 
HP Elite Dragonfly G4 13.5 inches core i5-1335u 16GB - 512GB iris xe graphics g7 (80eu)      1
Dell Precision 3480 14.0 inches core i5-1335u 16GB - 256GB iris xe graphics g7 (80eu)        1
HP Pavilion 15 15.6 inches core i5-1335u 32GB - 2000GB iris xe graphics g7 (80eu)            1
HP ProBook 450 G10 15.6 inches core i5-1335u 16GB - 4000GB iris xe graphics g7 (80eu)        1
HP 17 17.3 inches core i3-1115g4 3

In [8]:
#decrease the size of the inference dataset for compability with UI
data_for_inference=data_for_inference.drop_duplicates(subset=["name_to_display"])
X=laptop_data2.loc[data_for_inference.index] #create training data X
data_for_inference.reset_index(drop=True,inplace=True)
data_for_inference.shape

(9090, 67)

#### Recommendation based on the cost and purpose of the laptop

In [9]:
X=X[["Cost","Display Score","Portability Score","Play Score","Work Score"]]

In [10]:
X

,Cost,Display Score,Portability Score,Play Score,Work Score
0,2249.00,8.0,6.5,8.3,9.2
4,2138.99,8.0,6.5,8.2,8.8
8,3769.00,8.0,6.0,9.6,9.4
9,4658.99,8.0,6.0,9.6,9.2
12,3224.01,8.0,6.0,9.6,9.2
...,...,...,...,...,...
287826,719.00,1.5,7.3,1.9,5.0
288668,909.00,7.0,7.3,1.8,4.8
288731,849.00,7.0,7.3,1.8,4.6
288997,629.00,7.0,7.3,1.6,4.0


Normalization

In [11]:
# Normalize the data
normalized_X = (X - X.min()) / (X.max() - X.min())
normalized_X

,Cost,Display Score,Portability Score,Play Score,Work Score
0,0.174428,0.789474,0.65,0.833333,0.973684
4,0.165113,0.789474,0.65,0.822222,0.921053
8,0.303133,0.789474,0.60,0.977778,1.000000
9,0.378492,0.789474,0.60,0.977778,0.973684
12,0.256986,0.789474,0.60,0.977778,0.973684
...,...,...,...,...,...
287826,0.044877,0.105263,0.73,0.122222,0.421053
288668,0.060965,0.684211,0.73,0.111111,0.394737
288731,0.055885,0.684211,0.73,0.111111,0.368421
288997,0.037257,0.684211,0.73,0.088889,0.289474


KNN based on weighted euclidean distance

In [12]:
feature_weights = np.array([0.4, 0.05, 0.05,0.25,0.25])  #give cost, gaming score and work score more impact

# Function to calculate weighted Euclidean distance
def weighted_euclidean_distance(data, weights):
    n = data.shape[0]
    distances = np.zeros((n, n))  # Distance matrix
    for i in range(n):
        diff = (data - data.iloc[i]) * weights
        distances[i] = np.linalg.norm(diff, axis=1)
    return distances

# Compute weighted distance matrix
weighted_distances = weighted_euclidean_distance(normalized_X, feature_weights)

In [13]:
# Generate dictionary of top 10 closest neighbors for each laptop
top_neighbors_dict = {}
for i in range(len(weighted_distances)):
    neighbors = np.argsort(weighted_distances[i])[1:11]  
    top_neighbors_dict[i] = neighbors.tolist()
top_neighbors_dict

{0: [18, 34, 31, 166, 23, 194, 315, 20, 198, 541],
 1: [9, 15, 541, 34, 23, 20, 310, 225, 151, 530],
 2: [28, 50, 19, 356, 45, 228, 404, 26, 43, 308],
 3: [35, 48, 44, 99, 88, 32, 49, 170, 17, 24],
 4: [8, 267, 327, 265, 350, 354, 294, 353, 33, 37],
 5: [252, 358, 281, 253, 277, 312, 391, 255, 162, 226],
 6: [7, 29, 14, 141, 130, 298, 167, 138, 319, 10],
 7: [6, 29, 14, 141, 130, 298, 319, 249, 167, 138],
 8: [4, 354, 265, 327, 294, 267, 41, 350, 11, 274],
 9: [1, 541, 15, 34, 23, 20, 310, 225, 163, 151],
 10: [138, 402, 302, 167, 289, 298, 283, 277, 130, 281],
 11: [27, 33, 37, 8, 354, 4, 294, 72, 265, 327],
 12: [13, 77, 136, 130, 298, 10, 167, 141, 138, 547],
 13: [12, 77, 136, 130, 298, 141, 14, 276, 167, 10],
 14: [141, 7, 29, 319, 6, 130, 136, 249, 13, 298],
 15: [1, 9, 541, 34, 23, 20, 530, 310, 225, 151],
 16: [538, 530, 537, 20, 23, 31, 221, 540, 457, 536],
 17: [49, 88, 32, 22, 325, 140, 35, 126, 3, 48],
 18: [0, 34, 31, 315, 166, 194, 23, 296, 20, 5],
 19: [26, 45, 50, 28, 2

Store the inference data and the neighbour dictionary to serve the UI

In [14]:
with open("../data/small_lap_data.pkl","wb") as f:
    pkl.dump(data_for_inference,f) 

In [15]:
with open("../data/neighbours.pkl","wb") as f:
    pkl.dump(top_neighbors_dict,f) 

#### Inference

In [16]:
def find_nearest_neighbor(laptop_link):
    ind=data_for_inference[data_for_inference["link"]==laptop_link].index[0]
    return data_for_inference.loc[top_neighbors_dict[ind]][["link","name","Cost","Display Score","Portability Score","Play Score","Work Score"]]

In [17]:
link=data_for_inference.loc[300].link
print(link)
find_nearest_neighbor(link)

https://laptopmedia.com/laptop-specs/msi-raider-ge78-hx-71/


,link,name,Cost,Display Score,Portability Score,Play Score,Work Score
247,https://laptopmedia.com/laptop-specs/msi-vecto...,MSI Vector 17 HX,2714.00,8.5,6.0,8.7,8.9
255,https://laptopmedia.com/laptop-specs/msi-vecto...,MSI Vector 17 HX,2659.00,8.5,6.0,8.7,8.9
544,https://laptopmedia.com/laptop-specs/alienware...,Alienware m18 R2,2699.00,8.0,5.0,8.6,8.8
283,https://laptopmedia.com/laptop-specs/msi-raide...,MSI Raider GE78 HX,2759.00,8.5,5.7,8.7,9.1
277,https://laptopmedia.com/laptop-specs/msi-vecto...,MSI Vector 17 HX,2719.00,8.5,6.0,8.7,9.1
281,https://laptopmedia.com/laptop-specs/msi-vecto...,MSI Vector 17 HX,2669.00,8.5,6.0,8.7,9.1
253,https://laptopmedia.com/laptop-specs/msi-vecto...,MSI Vector 17 HX,2514.00,8.5,6.0,8.7,8.9
358,https://laptopmedia.com/laptop-specs/msi-raide...,MSI Raider GE78,2625.00,8.5,5.7,8.7,9.1
289,https://laptopmedia.com/laptop-specs/msi-raide...,MSI Raider GE78 HX,2819.00,8.5,5.7,8.7,9.1
455,https://laptopmedia.com/laptop-specs/lenovo-le...,Lenovo Legion Pro 5i 16,2619.99,8.5,4.4,8.6,8.9
